## Loading the Database
 

In [25]:
import sys
import os

import mysql.connector
from mysql.connector import Error
import json
from datetime import datetime, timedelta

# Add the project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
if project_root not in sys.path:
    sys.path.append(project_root)
    
from app.database import create_connection  # Replace `function_name` with the specific function you want to import

connection = create_connection()

## Insert Buildings, rooms, and avaiability


In [19]:

def load_buildings(connection, buildings):
    """
    Inserts building data into the buildings table.
    :param connection: MySQL connection
    :param buildings: Dictionary of building codes and names
    """
    cursor = connection.cursor()
    try:
        for code, name in buildings.items():
            cursor.execute("""
                INSERT IGNORE INTO buildings (building_code, building_name)
                VALUES (%s, %s);
            """, (code, name))
        connection.commit()
        print("Buildings loaded successfully!")
    except mysql.connector.Error as e:
        print(f"Error loading buildings: {e}")
    finally:
        cursor.close()



# Load buildings
buildings = {
    "ACE": "Accolade Building East",
    "ACW": "Accolade Building West",
    "AO": "Archives of Ontario",
    "ATK": "Atkinson",
    "BC": "Norman Bethune College",
    "BCSS": "Bennett Centre for Student Services",
    "BRG": "Bergeron Centre for Engineering Excellence",
    "BSB": "Behavioural Sciences Building",
    "BU": "Burton Auditorium",
    "CB": "Chemistry Building",
    "CC": "Calumet College",
    "CFA": "The Joan & Martin Goldfarb Centre for Fine Arts",
    "CFT": "Centre for Film and Theatre",
    "CLH": "Curtis Lecture Halls",
    "CMB": "Computer Methods Building",
    "CSQ": "Central Square",
    "CUB": "Central Utilities Building",
    "DB": "Dahdaleh Building (formerly TEL)",
    "ELC": "Executive Learning Centre",
    "FC": "Founders College",
    "FL": "Frost Library (Glendon campus)",
    "FRQ": "Farquharson Life Sciences",
    "FTC": "Founders Tennis Court",
    "GH": "Glendon Hall (Glendon campus)",
    "HC": "Lorna R. Marsden Honours Court & Welcome Centre",
    "HH": "Hart House (Osgoode Hall Law School)",
    "HNE": "Health, Nursing and Environmental Studies Building",
    "HR": "Hilliard Residence (Glendon campus)",
    "IKB": "Ignat Kaneff Building (Osgoode Hall Law School)",
    "K": "Kinsmen Building",
    "KT": "Kaneff Tower",
    "LAS": "Lassonde Building",
    "LMP": "LA&PS @ IBM Markham",
    "LSB": "Life Sciences Building",
    "LUM": "Lumbers Building",
    "MB": "Rob and Cheryl McEwen Graduate Study & Research Building",
    "MC": "McLaughlin College",
    "OC": "Off Campus (contact the department that offers the course)",
    "PR": "Physical Resources Building",
    "PSE": "Petrie Science and Engineering Building / Petrie Observatory",
    "R N": "Ross Building - North wing",
    "R S": "Ross Building - South wing",
    "SAY": "Seneca @ York",
    "SC": "Stong College",
    "SCL": "Scott Library",
    "SHR": "Sherman Health Science Research Centre",
    "SLH": "Stedman Lecture Halls",
    "SSB": "Seymour Schulich Building",
    "ST": "Sheridan College - Trafalgar Campus",
    "STC": "Student Centre",
    "STL": "Steacie Science and Engineering Library",
    "TC Sobeys": "Tennis Canada (TC Sobeys)",
    "TEL": "Technology and Enhanced Learning Building (effective summer 2016, Dahdaleh Building, DB)",
    "TFC": "Track and Field Centre",
    "TM Tait": "Tait McKenzie Centre (East/West)",
    "TTC": "Tait Tennis Courts",
    "VC": "Vanier College",
    "VH": "Vari Hall",
    "WC": "Winters College",
    "WOB": "West Office Building",
    "WSC": "William Small Centre",
    "YH": "York Hall (Glendon campus)",
    "YL": "York Lanes",
    "YSF FLDHSE": "York Student Fieldhouse (East/West) (Tait McKenzie Building)",
    "YSF FLDHSEE": "York Student Fieldhouse (East side only) (Tait McKenzie Building)",
    "YSF FLDHSEW": "York Student Fieldhouse (West side only) (Tait McKenzie Building)",
}
load_buildings(connection, buildings)
print("success")



Buildings loaded successfully!
success


In [39]:
from datetime import datetime, timedelta

def load_rooms_and_availabilities(connection, data):
    """
    Inserts rooms into the database and calculates unoccupied time slots (availability).
    Handles full-day availability if a room is unoccupied for a given day.
    Includes all days from Monday to Sunday.
    :param connection: Active MySQL connection.
    :param data: Dictionary of room_name -> schedules.
    """
    try:
        cursor = connection.cursor()

        # Operating hours: 8:30 AM to 10:00 PM
        opening_time = datetime.strptime("08:30", "%H:%M")
        closing_time = datetime.strptime("22:00", "%H:%M")

        valid_days = {"M", "T", "W", "R", "F", "S", "U"}  # Include all days of the week

        for room_name, schedules in data.items():
            # Split the room name to get building code and room number
            parts = room_name.split(" ", 1)
            if len(parts) < 2:
                print(f"Invalid room name format: {room_name}. Skipping.")
                continue
            building_code, room_number = parts[0], parts[1]

            # Find building_id using building_code
            cursor.execute("SELECT building_id FROM buildings WHERE building_code = %s", (building_code,))
            result = cursor.fetchone()
            if not result:
                print(f"Building not found for code {building_code}. Skipping.")
                continue

            building_id = result[0]

            # Insert room into database
            cursor.execute("""
                INSERT INTO rooms (building_id, room_name)
                VALUES (%s, %s);
            """, (building_id, room_name))
            room_id = cursor.lastrowid
            print(f"Inserted room: {room_name}, ID: {room_id}")

            # Group occupied time slots by day
            occupied_slots = {day: [] for day in valid_days}
            for schedule in schedules:
                day = schedule.get("day")
                if day not in valid_days:
                    print(f"Invalid day code: {day}. Skipping.")
                    continue

                start_time = datetime.strptime(schedule["time"], "%H:%M")
                duration_minutes = int(schedule["duration"])
                end_time = start_time + timedelta(minutes=duration_minutes)
                occupied_slots[day].append((start_time, end_time))

            # Calculate unoccupied time slots for each day
            for day in valid_days:
                slots = occupied_slots[day]
                if not slots:
                    # Room is completely free for the entire day
                    cursor.execute("""
                        INSERT INTO availabilities (room_id, start_time, end_time, day)
                        VALUES (%s, %s, %s, %s);
                    """, (
                        room_id,
                        opening_time.strftime("%H:%M:%S"),
                        closing_time.strftime("%H:%M:%S"),
                        day
                    ))
                    print(f"Inserted full-day free slot: Room {room_name}, Day: {day}, Start: {opening_time}, End: {closing_time}")
                else:
                    # Sort occupied slots by start time
                    slots.sort(key=lambda x: x[0])
                    current_time = opening_time

                    for start, end in slots:
                        if current_time < start:
                            # Add unoccupied time before the next occupied slot
                            cursor.execute("""
                                INSERT INTO availabilities (room_id, start_time, end_time, day)
                                VALUES (%s, %s, %s, %s);
                            """, (
                                room_id,
                                current_time.strftime("%H:%M:%S"),
                                start.strftime("%H:%M:%S"),
                                day
                            ))
                            print(f"Inserted free slot: Room {room_name}, Day: {day}, Start: {current_time}, End: {start}")
                        current_time = max(current_time, end)

                    # Add remaining unoccupied time after the last occupied slot
                    if current_time < closing_time:
                        cursor.execute("""
                            INSERT INTO availabilities (room_id, start_time, end_time, day)
                            VALUES (%s, %s, %s, %s);
                        """, (
                            room_id,
                            current_time.strftime("%H:%M:%S"),
                            closing_time.strftime("%H:%M:%S"),
                            day
                        ))
                        print(f"Inserted free slot: Room {room_name}, Day: {day}, Start: {current_time}, End: {closing_time}")

        connection.commit()
        print("Rooms and availabilities loaded successfully.")
    except Exception as e:
        print(f"Error loading rooms and availabilities: {e}")
    finally:
        cursor.close()


In [40]:
# Establish connection
connection = create_connection()

# Load rooms and availabilities from JSON
import json
with open("../scraped backup/emptyrooms_fw24/all_rooms_2024_09_11.json", "r") as f:
    room_data = json.load(f)

load_rooms_and_availabilities(connection, room_data)




Inserted room: WC 117, ID: 2376
Inserted free slot: Room WC 117, Day: M, Start: 1900-01-01 08:30:00, End: 1900-01-01 10:30:00
Inserted free slot: Room WC 117, Day: M, Start: 1900-01-01 11:30:00, End: 1900-01-01 14:30:00
Inserted free slot: Room WC 117, Day: M, Start: 1900-01-01 17:30:00, End: 1900-01-01 22:00:00
Inserted free slot: Room WC 117, Day: W, Start: 1900-01-01 08:30:00, End: 1900-01-01 17:30:00
Inserted free slot: Room WC 117, Day: W, Start: 1900-01-01 20:30:00, End: 1900-01-01 22:00:00
Inserted free slot: Room WC 117, Day: F, Start: 1900-01-01 11:30:00, End: 1900-01-01 13:30:00
Inserted free slot: Room WC 117, Day: F, Start: 1900-01-01 14:30:00, End: 1900-01-01 22:00:00
Inserted free slot: Room WC 117, Day: T, Start: 1900-01-01 08:30:00, End: 1900-01-01 10:30:00
Inserted free slot: Room WC 117, Day: T, Start: 1900-01-01 11:30:00, End: 1900-01-01 14:30:00
Inserted free slot: Room WC 117, Day: T, Start: 1900-01-01 17:30:00, End: 1900-01-01 22:00:00
Inserted free slot: Room WC 

# Testing Cafes and Libraries

In [ ]:
import sys
import os

import mysql.connector
from mysql.connector import Error
import json
from datetime import datetime, timedelta

# Add the project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
if project_root not in sys.path:
    sys.path.append(project_root)

from database import get_cafes, get_libraries

cafes = get_cafes()

print(cafes)

libraries = get_libraries()

print(libraries)
